<h1>Data Analysis</h1>

<h3>File handling</h3>

<h4>Excel</h4>

In [3]:
from xlrd import open_workbook
import xlwt

with open_workbook('data/files/subvenciones.xls', on_demand=True) as book:
    associations = {}
    for sheet_name in book.sheet_names():
        sheet = book.sheet_by_name(sheet_name)
        for i in range(1, sheet.nrows):
            row = sheet.row(i)
            center = row[0].value
            subvention = row[2].value

            if center in associations:
                associations[center] = associations[center] + subvention
            else:
                associations[center] = subvention

associations

{'AMPA ANTONIO MACHADO': 2344.99,
 'AMPA BACHILLER ALONSO LOPEZ': 3200.0,
 'AMPA CASTILLA': 2604.44,
 'AMPA DAOIZ Y VELARDE': 3152.74,
 'AMPA EMILIO CASADO': 3015.67,
 'AMPA FEDERICO GARCIA LORCA': 1919.06,
 'AMPA GABRIEL Y GALAN': 2741.51,
 'AMPA LUIS BUÑUEL': 2081.0,
 'AMPA MIGUEL HERNANDEZ': 2923.35,
 'AMPA MIRAFLORES': 2787.21,
 'AMPA PARQUE CATALUÑA': 2604.44,
 'AMPA PROFESOR TIERNO GALVÁN': 1286.0,
 'AMPA SEIS DE DICIEMBRE': 1950.0,
 'AMPA VALDEPALITOS': 3929.5,
 'AMPA LA CHOPERA': 1430.0,
 'AMPA EL CUQUILLO': 1507.83,
 'AMPA VALDELAPARRA': 2465.0,
 'AMPA RIVENDEL': 2200.0,
 'AMPA AGORA': 2421.67,
 'AMPA ALDEBARAN': 3107.05,
 'AMPA GINER DE LOS RIOS': 2058.0,
 'AMPA SEVERO OCHOA': 3563.9700000000003,
 'AMPA VIRGEN DE LA PAZ': 1416.45,
 'AMPA JUAN XXIII': 1781.98,
 'AMPA SAN ANTONIO': 2101.83,
 'AMPA PADRE  MANYANET': 2695.82,
 'AMPA FAPA': 3198.43}

In [4]:
with open_workbook('data/files/subvenciones.xls', on_demand=True) as book_r:
    associations = {}
    book_w = xlwt.Workbook()
    for name in book_r.sheet_names():
        sheet_r = book_r.sheet_by_name(name)
        sheet_w = book_w.add_sheet(name)
        for i in range(sheet_r.nrows):
            for j in range(sheet_r.ncols):
                sheet_w.write(i, j, sheet_r.row(i)[j].value)
            if i != 0:
                row = sheet_r.row(i)
                center = row[0].value
                subvention = float(row[2].value)
                if center in associations:
                    associations[center] = associations[center] + subvention
                else:
                    associations[center] = subvention

    sheet_w = book_w.add_sheet('Totales')
    sheet_w.write(0, 0, "Asociación")
    sheet_w.write(0, 1, "Importe total")
    sheet_w.write(0, 2, "Importe justificado")
    sheet_w.write(0, 3, "Restante")

    for i, clave in enumerate(associations):
        row = i + 1
        row_form = i + 2
        sheet_w.write(row, 0, clave)
        sheet_w.write(row, 1, associations[clave])
        sheet_w.write(row, 2, 0)
        sheet_w.write(row, 3, xlwt.Formula("C" + str(row_form) + "-" + "B" + str(row_form)))

    book_w.save('data/files/subvenciones_totales.xls')

<h4>XML</h4>

In [5]:
import xml.etree.ElementTree as ET

In [6]:
tree = ET.parse('data/files/subvenciones.xml')
root = tree.getroot()
associations = {}
for row in root:
    center = row[0].text
    subvention = float(row[2].text)
    
    if center in associations:
        associations[center] = associations[center] + subvention
    else:
        associations[center] = subvention
associations

{'AMPA ANTONIO MACHADO': 2344.99,
 'AMPA BACHILLER ALONSO LOPEZ': 3200.0,
 'AMPA CASTILLA': 2604.44,
 'AMPA DAOIZ Y VELARDE': 3152.74,
 'AMPA EMILIO CASADO': 3015.67,
 'AMPA FEDERICO GARCIA LORCA': 1919.06,
 'AMPA GABRIEL Y GALAN': 2741.51,
 'AMPA LUIS BUÑUEL': 2081.0,
 'AMPA MIGUEL HERNANDEZ': 2923.35,
 'AMPA MIRAFLORES': 2787.21,
 'AMPA PARQUE CATALUÑA': 2604.44,
 'AMPA PROFESOR TIERNO GALVÁN': 1286.0,
 'AMPA SEIS DE DICIEMBRE': 1950.0,
 'AMPA VALDEPALITOS': 3929.5,
 'AMPA LA CHOPERA': 1430.0,
 'AMPA EL CUQUILLO': 1507.83,
 'AMPA VALDELAPARRA': 2465.0,
 'AMPA RIVENDEL': 2200.0,
 'AMPA AGORA': 2421.67,
 'AMPA ALDEBARAN': 3107.05,
 'AMPA GINER DE LOS RIOS': 2058.0,
 'AMPA SEVERO OCHOA': 3563.9700000000003,
 'AMPA VIRGEN DE LA PAZ': 1416.45,
 'AMPA JUAN XXIII': 1781.98,
 'AMPA SAN ANTONIO': 2101.83,
 'AMPA PADRE  MANYANET': 2695.82,
 'AMPA FAPA': 3198.43}

In [7]:
tree = ET.parse('data/files/subvenciones.xml')
root = tree.getroot()
new = ET.ElementTree()
new_root = ET.Element("root")
new._setroot(new_root)
current_element = ET.Element("Asociacion")
current_asociation = ""
activities = ET.SubElement(current_element, "activities")
cost = 0
for row in root.findall('Row'):
    asoc = row.find('Asociaci_n').text
    act = row.find('Actividad_Subvencionada').text
    imp = float(row.find('Importe').text)
    if current_asociation != asoc:
        total_cost = ET.SubElement(current_element, "Total")
        total_cost.text = str(cost)
        current_element = ET.SubElement(new_root, "Asociacion")
        current_element.set('nombre', asoc)
        activities = ET.SubElement(current_element, "activities")
        cost = 0
    act_elem = ET.SubElement(activities, "Actividad")
    nom_elem = ET.SubElement(act_elem, "Nombre")
    nom_elem.text = act
    imp_elem = ET.SubElement(act_elem, "Subvencion")
    imp_elem.text = str(imp)
    cost = cost + imp
    current_asociation = asoc
new.write('data/files/subvenciones_totales.xml')

<h4>JSON</h4>

In [8]:
import json

In [9]:
with open('data/files/subvenciones.json') as json_subventions:
    subvention = json.load(json_subventions)
    
subvention

[{'Asociación': 'AMPA ANTONIO MACHADO',
  'Actividad Subvencionada': 'TALLER FIESTA DE CARNAVAL',
  'Importe en euros': '94.56'},
 {'Asociación': 'AMPA ANTONIO MACHADO',
  'Actividad Subvencionada': 'TALLER DIA DEL PADRE',
  'Importe en euros': '39.04 '},
 {'Asociación': 'AMPA ANTONIO MACHADO',
  'Actividad Subvencionada': 'TALLER DIA DE LA MADRE',
  'Importe en euros': '43.64 '},
 {'Asociación': 'AMPA ANTONIO MACHADO',
  'Actividad Subvencionada': 'FIESTA FIN DE CURSO',
  'Importe en euros': '921.00 '},
 {'Asociación': 'AMPA ANTONIO MACHADO',
  'Actividad Subvencionada': 'CONCURSO LOGOTIPO AMPA',
  'Importe en euros': '56.57 '},
 {'Asociación': 'AMPA ANTONIO MACHADO',
  'Actividad Subvencionada': 'ASOCIACION FAPA ALCOBENDAS',
  'Importe en euros': '80.00 '},
 {'Asociación': 'AMPA ANTONIO MACHADO',
  'Actividad Subvencionada': 'ASOCIACION FAPA GINER DE LOS RIOS',
  'Importe en euros': '86.79 '},
 {'Asociación': 'AMPA ANTONIO MACHADO',
  'Actividad Subvencionada': 'FIESTA DE NAVIDAD',
 

In [10]:
with open('data/files/subvenciones.json', encoding='utf-8') as json_subventions, open('data/files/subvenciones_totales.json', encoding='utf-8', mode='w') as json_w:
    subventions = json.load(json_subventions)

    total_obj = {}

    for subvention in subventions:
        center = subvention['Asociación']
        activity = subvention['Actividad Subvencionada']
        cost = float(subvention['Importe en euros'].replace('.', ''))

        if center not in total_obj:
            total_obj[center] = {'activities': [activity], 'cost': cost}

        total_obj[center]['activities'].append(activity)
        total_obj[center]['cost'] += cost

    json.dump(total_obj, json_w, ensure_ascii=False, indent=4)

<h3>CSV</h3>

In [11]:
import csv

In [12]:
with open('data/files/subvenciones.csv', encoding='latin1') as csv_subvenciones:
    reader = csv.reader(csv_subvenciones)
    next(reader, None)
    total_cost = 0
    for row in reader:
        association = row[0]
        cost = float(row[2])

        total_cost += cost

total_cost

66487.94

In [13]:
with open('data/files/subvenciones.csv', encoding='latin1') as csv_subvenciones:
    reader = csv.reader(csv_subvenciones)
    next(reader, None)

    total_csv_obj = {}
    for row in reader:
        association = row[0]
        cost = float(row[2])

        if association not in total_csv_obj:
            total_csv_obj[association] = cost
        
        total_csv_obj[association] = total_csv_obj[association] + cost

total_csv_obj

{'AMPA ANTONIO MACHADO': 2439.5499999999997,
 'AMPA BACHILLER ALONSO LOPEZ': 3680.0,
 'AMPA CASTILLA': 2728.44,
 'AMPA DAOIZ Y VELARDE': 3522.74,
 'AMPA EMILIO CASADO': 3311.34,
 'AMPA FEDERICO GARCIA LORCA': 3319.06,
 'AMPA GABRIEL Y GALAN': 3893.02,
 'AMPA LUIS BUÑUEL': 2171.0,
 'AMPA MIGUEL HERNANDEZ': 3223.35,
 'AMPA MIRAFLORES': 4287.21,
 'AMPA PARQUE CATALUÑA': 2894.44,
 'AMPA PROFESOR TIERNO GALVÁN': 1586.0,
 'AMPA SEIS DE DICIEMBRE': 2426.0,
 'AMPA VALDEPALITOS': 4315.0,
 'AMPA LA CHOPERA': 2420.0,
 'AMPA EL CUQUILLO': 2117.83,
 'AMPA VALDELAPARRA': 3150.0,
 'AMPA RIVENDEL': 2430.0,
 'AMPA AGORA': 2923.34,
 'AMPA ALDEBARAN': 4087.1,
 'AMPA GINER DE LOS RIOS': 2731.0,
 'AMPA SEVERO OCHOA': 4545.9400000000005,
 'AMPA VIRGEN DE LA PAZ': 2343.9,
 'AMPA JUAN XXIII': 2581.98,
 'AMPA SAN ANTONIO': 2996.83,
 'AMPA PADRE  MANYANET': 3495.82,
 'AMPA FAPA': 3918.43}

In [14]:
with open('data/files/subvenciones.csv', encoding='latin1') as csv_subv_r, open('data/files/subvenciones_totales.csv', encoding='latin1', mode='w') as csv_subv_w:
    dict_reader = csv.DictReader(csv_subv_r)

    headers = dict_reader.fieldnames + ['Justificacion requerida']

    dict_writer = csv.DictWriter(csv_subv_w, fieldnames=headers)

    dict_writer.writeheader()

    for row in dict_reader:
        cost = float(row['Importe'])
        if cost > 300:
            row['Justificacion requerida'] = "Si"
        else:
            row['Justificacion requerida'] = "No"
        
        dict_writer.writerow(row)